## Дз 3

### Easy
#### 1. Загрузка данных

In [2]:
import csv
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from matplotlib.colors import ListedColormap
import seaborn as sns

In [3]:
data = pd.read_pickle('../data/train.pickle')

x = data['features']
y = data['labels']
x = x.reshape(x.shape[0], -1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

#### 2. Обучение модели

In [4]:
model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)

C:\Users\lizag\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

#### 3. Оценка качества  

In [5]:
y_pred = model.predict(x_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred, average='macro')}")
print(f"Recall: {recall_score(y_test, y_pred, average='macro')}")

Accuracy: 0.9517241379310345
Precision: 0.9521229351573929
Recall: 0.9460699305457236


Accuracy дает общее представление о правильном классифицировании дорожных знаков. Если распределение классов примерно сбалансировано, то она является хорошим показателем

#### 4. Анализ ошибок 

In [6]:
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix.shape

(43, 43)

In [7]:
label_names_df = pd.read_csv("../data/label_names.csv")
errors = [(conf_matrix[i, j], i, j) for i in range(43) for j in range(43) if i != j]  # список ошибок с индексами 
most_error = sorted(errors, key=lambda x: x[0], reverse=True)[:15]  # сортируем, берем первые 15 
for i in most_error:
    most_error_name = (label_names_df['SignName'].iloc[i[1]], label_names_df['SignName'].iloc[i[2]])
    print(most_error_name)

('Speed limit (80km/h)', 'Speed limit (60km/h)')
('Speed limit (60km/h)', 'Speed limit (80km/h)')
('Speed limit (50km/h)', 'Speed limit (30km/h)')
('Speed limit (30km/h)', 'Speed limit (50km/h)')
('Speed limit (50km/h)', 'Speed limit (80km/h)')
('Speed limit (30km/h)', 'Speed limit (70km/h)')
('Speed limit (70km/h)', 'Speed limit (50km/h)')
('Stop', 'No entry')
('Speed limit (70km/h)', 'Speed limit (80km/h)')
('Slippery road', 'Children crossing')
('Speed limit (30km/h)', 'Speed limit (80km/h)')
('Speed limit (30km/h)', 'Speed limit (100km/h)')
('Speed limit (50km/h)', 'Speed limit (70km/h)')
('Speed limit (50km/h)', 'Speed limit (100km/h)')
('Speed limit (80km/h)', 'Speed limit (30km/h)')


Вывод: модель плохо распознает знаки скоростного ограничения

### Medium 
#### 1. Обучение моделей

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
log_reg = LogisticRegression(max_iter=300)
knn = KNeighborsClassifier()
nb = GaussianNB()
svm_lin = SVC(kernel='linear', max_iter=300)
svm_rbf = SVC(kernel='rbf', max_iter=300)
svm_poly = SVC(kernel='poly', max_iter=300)
models = {'Logistic Regression': log_reg,
          'knn': knn,
          'NB': nb,
          'SVM_lin': svm_lin,
          'SVM_rbf': svm_rbf,
          'SVM_poly': svm_poly}
for name, model in models.items():
    model.fit(x_train, y_train)

C:\Users\lizag\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\lizag\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\lizag\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\lizag\anaconda3\Lib\site-packages\sklearn

#### 2. Оценка работы моделей 

In [12]:
results = {}
for name, model in models.items():
    y_pred = model.predict(x_test)
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_test, y_pred, average='weighted'),
        'f1': f1_score(y_test, y_pred, average='weighted', zero_division=0)
    }
    results[name] = metrics

for name, metrics in results.items():
    print(f"{name}: accuracy: {metrics['accuracy']:.4f}, precision: {metrics['precision']:.4f}, "
          f"recall: {metrics['recall']:.4f}, f1: {metrics['f1']:.4f}")

C:\Users\lizag\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Logistic Regression: accuracy: 0.9556, precision: 0.9558, recall: 0.9556, f1: 0.9555
knn: accuracy: 0.8787, precision: 0.8842, recall: 0.8787, f1: 0.8797
NB: accuracy: 0.1716, precision: 0.4204, recall: 0.1716, f1: 0.1803
SVM_lin: accuracy: 0.8861, precision: 0.8897, recall: 0.8861, f1: 0.8866
SVM_rbf: accuracy: 0.7570, precision: 0.7929, recall: 0.7570, f1: 0.7594
SVM_poly: accuracy: 0.5122, precision: 0.6745, recall: 0.5122, f1: 0.5304


#### 3. Выводы 

1. Логическая регрессия и `SVM` с линейным ядром показали лучшие результаты. При этом `SVM` с ядрами `rbf`, `poly` работают не так хорошо и довольно медленно на большом объеме входных данных 
2. Производительность `knn` зависит от выбранных параметров, способа измерения расстояния и размеров изображений
3. Модель `NB` хорошо работает если признаки независимы, но для изображений это выполняется редко, поэтому эта модель имеет низкую точность по сравнению с другими 
